In [21]:
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn.interfaces.fmriprep import load_confounds
import numpy as np
from nilearn.image import resample_img


In [22]:
import json
fmri_filename = "/mnt/NAS2-2/data/SAD_gangnam_resting_2/fMRIPrep_total/sub-c0009/ses-01/func/sub-c0009_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
# Assuming your fMRI filename is fmri_filename
# Construct the path to the JSON sidecar file
json_filename = "/mnt/NAS2-2/data/SAD_gangnam_resting_2/fMRIPrep_total/sub-c0009/ses-01/func/sub-c0009_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.json"

# Load the JSON file
with open(json_filename, 'r') as f:
    metadata = json.load(f)
#print(metadata)
# Extract the TR
tr = metadata['RepetitionTime']  # TR is in seconds

print(f"The TR is: {tr} seconds")

The TR is: 2 seconds


In [26]:
# Step 1: Import necessary libraries
import nibabel as nib
from nilearn import image
from nilearn.interfaces.fmriprep import load_confounds
from nilearn.maskers import NiftiMasker

# Step 2: Load your fMRI data
fmri_filename = "/mnt/NAS2-2/data/SAD_gangnam_resting_2/fMRIPrep_total/sub-c0009/ses-01/func/sub-c0009_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
fmri_img = image.load_img(fmri_filename)

# Step 3: Apply 8mm smoothing to the original image
fmri_smoothed_img = image.smooth_img(fmri_img, fwhm=8)

# Step 4: Resample the smoothed image to 4x4x4 mm voxel size
fmri_resampled_img = resample_img(fmri_smoothed_img, target_affine=np.diag((4,4,4)))

# Step 5: Load the confounds using load_confounds
confounds, sample_mask = load_confounds_strategy(
    fmri_filename, denoise_strategy="scrubbing", global_signal="basic", fd_threshold=0.5, std_dvars_threshold=1.5
)


# Step 7: Discard the first four volumes
# Adjust the sample_mask to exclude the first four volumes
if sample_mask is None:
    sample_mask = np.arange(fmri_resampled_img.shape[-1])  # Include all volumes
# Exclude the first four volumes
sample_mask = sample_mask[sample_mask >= 4]

# Step 8: Create a NiftiMasker without additional smoothing
masker = NiftiMasker(
    smoothing_fwhm=None,  # No additional smoothing since it's already applied
    standardize=False,
    detrend=False,
    high_pass=None,       # No additional high-pass filtering
    low_pass=0.1,         # Apply low-pass filter at 0.1 Hz
    t_r=tr,               # Use the TR extracted from the metadata
)

# Step 9: Fit the masker (this computes the mask)
masker.fit(fmri_resampled_img)

# Step 10: Apply the masker to perform confound regression and filtering
fmri_denoised = masker.transform(
    fmri_resampled_img,
    confounds=confounds,
    sample_mask=sample_mask
)

# Step 11: Reconstruct the 4D denoised image from the 2D data matrix
fmri_denoised_img = masker.inverse_transform(fmri_denoised)


ValueError: This function can only deal with 3D images

In [25]:
output_filename = '/mnt/NAS2-2/data/SAD_gangnam_resting_2/fMRIPrep_total/sub-c0009/ses-01/func/sub-c0009_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-smoothed8mm_resampled4mm_scrubbing_bold.nii.gz'
fmri_denoised_img.to_filename(output_filename)